In [70]:
from __future__ import division, unicode_literals

import pandas as pd
import numpy as np

import dateutil
import datetime
import time
from time import sleep

from bs4 import BeautifulSoup
import requests

import re
import html5lib

import pickle

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 25)
pd.set_option('display.precision', 3)

In [149]:
main_url='https://www.boxofficemojo.com'
genre_page_url='https://www.boxofficemojo.com/genres/'

In [128]:
# import re
# animation_genre_regex=re.compile('anim')
# animation_genre_regex
# genre_soup.find_all('a')

In [146]:
# make a soup of data out of a url
def make_soup(url):
    # request data from url
    response=requests.get(url)
    # get html data 
    page=response.text
    # parse html data into lxml
    soup=BeautifulSoup(page,'lxml')
    return soup

In [161]:
# get links to 8 subgenres pages under 'Animation'
animation_subgenre_landing_urls = []
genre_soup=make_soup(genre_page_url)
for link in genre_soup.find_all('a'):
    if ('chart' in str(link)) & ('Animation' in str(link)):
        animation_subgenre_landing_urls.append(genre_page_url+link['href'][2:])

animation_subgenre_landing_urls


['https://www.boxofficemojo.com/genres/chart/?id=animation.htm',
 'https://www.boxofficemojo.com/genres/chart/?id=anime.htm',
 'https://www.boxofficemojo.com/genres/chart/?id=computeranimation.htm',
 'https://www.boxofficemojo.com/genres/chart/?id=animatedfantasy.htm',
 'https://www.boxofficemojo.com/genres/chart/?id=animationmotioncap.htm',
 'https://www.boxofficemojo.com/genres/chart/?id=scifianimation.htm',
 'https://www.boxofficemojo.com/genres/chart/?id=animationstop.htm',
 'https://www.boxofficemojo.com/genres/chart/?id=basedonanimatedtv.htm']

In [ ]:
# from each subgenre landing page, get links to each additional page
# example a-tag: <a href="/genres/chart/?view=main&amp;sort=gross&amp;order=DESC&amp-
# ;pagenum=2&amp;id=anime.htm">#101-116</a>
# href contains 'chart' & 'pagenum='
subgenre_page_urls=list(animation_subgenre_landing_urls)
for url in animation_subgenre_landing_urls:
    subgenre_landing_soup=make_soup(url)
    for link in subgenre_landing_soup.find_all('a'):
        if ('chart' in str(link)) & ('pagenum='in str(link)):
            subgenre_page_urls.append(main_url+link['href'])

# remove duplicates
subgenre_page_urls=list(set(subgenre_page_urls))

In [192]:
len(subgenre_page_urls)

15

Iterate through the links one at a time.  
* Make a new request for the link  
* Turn the link into "soup"  
* Extract the links from the link  
* Store the address in the list  addr  

In [114]:
# url1 = 'https://www.boxofficemojo.com/genres/chart/?id=animation.htm'

In [190]:
# get movie links
movie_links=[]
for url in subgenre_page_urls:
    page_soup=make_soup(url)
    for link in page_soup.find('h1').findNextSibling('table').find_all('a'):
            if '/movies/?id=' in str(link):
                movie_links.append(main_url+link['href'])
    print(len(movie_links),url)
                
len(movie_links)

70 https://www.boxofficemojo.com/genres/chart/?id=animatedfantasy.htm
170 https://www.boxofficemojo.com/genres/chart/?id=animation.htm
177 https://www.boxofficemojo.com/genres/chart/?id=animationmotioncap.htm
204 https://www.boxofficemojo.com/genres/chart/?id=animationstop.htm
304 https://www.boxofficemojo.com/genres/chart/?id=anime.htm
348 https://www.boxofficemojo.com/genres/chart/?id=basedonanimatedtv.htm
448 https://www.boxofficemojo.com/genres/chart/?id=computeranimation.htm
496 https://www.boxofficemojo.com/genres/chart/?id=scifianimation.htm
596 https://www.boxofficemojo.com/genres/chart/?view=main&sort=gross&order=DESC&pagenum=2&id=animation.htm
612 https://www.boxofficemojo.com/genres/chart/?view=main&sort=gross&order=DESC&pagenum=2&id=anime.htm
696 https://www.boxofficemojo.com/genres/chart/?view=main&sort=gross&order=DESC&pagenum=2&id=computeranimation.htm
796 https://www.boxofficemojo.com/genres/chart/?view=main&sort=gross&order=DESC&pagenum=3&id=animation.htm
896 https://w

1039

In [191]:
# sanity check if number of movies scrapped matches records on the 
# genre index page: https://www.boxofficemojo.com/genres/
# n_movies_scrapped = len(get_movie_urls(animation_subgenre_urls)) 
526+115+180+70+7+48+27+44

1017

> Scrapped 1039 movie links.  
> Notice that some don't have a rank, which causes discrepancy from official movie counts (1017) on [Genre Index](https://www.boxofficemojo.com/genres/) page 